In [ ]:
df = pd.read_csv('/home/billbrod/Data/spatial_frequency_preferences/derivatives/tuning_curves_summary/stim_class/posterior/v1-2-3_e1-12_eccen_bin_hemi_bin_tuning_curves_full.csv')

In [ ]:
tmp_df = df[(df.subject=='sub-wlsubj001')&(df.session=='ses-01')&(df.frequency_type=='Local spatial frequency (cpd)')&(df.varea==1)&(df.eccen==1.5)]

In [ ]:
pal = [sns.color_palette('deep', 4)[0]] + [sns.color_palette('deep', 4)[2]]
with sns.plotting_context('poster', font_scale=1.75):
    g = sfp.summary_plots.main(df, row=None, col='subject', task='task-sfp', session=['ses-01', 'ses-02', 'ses-03'], y='preferred_period', varea=1, frequency_type='Local spatial frequency (cpd)', eccen_range=(0, 11),
                           plot_func=[sfp.plotting.plot_median_fit, sfp.plotting.scatter_ci_dist], xlim=(0,12), ylim=(0,4), save_path='results.pdf', col_wrap=4, axes_style='white', x_jitter=[None, .2],
                           col_order=['sub-wlsubj001', 'sub-wlsubj045', 'sub-wlsubj014', 'sub-wlsubj004'], size=6, palette=pal, stimulus_superclass=['circular', 'radial'])
    g.set_ylabels('Preferred period (dpc)')
    g.set_xlabels('Eccentricity (deg)')
    g.set_yticklabels([0, '', 1, '', 2, '', 3, '', 4])
    g._legend.set_title("Stimulus class")
    for i, ax in enumerate(g.axes.flatten()):
        ax.set_title('Subject %s' % (i+1))
    g.savefig('results.pdf')

In [ ]:
with sns.plotting_context('poster', font_scale=1.75):
    g = sfp.summary_plots.main(df, row=None, col='subject', task='task-sfp', session=['ses-01', 'ses-02', 'ses-03'], y='preferred_period', varea=1, frequency_type='Local spatial frequency (cpd)', eccen_range=(1, 11),
                               plot_func=[sfp.plotting.plot_median_fit, sfp.plotting.scatter_ci_dist], xlim=(0,12), ylim=(0,4), save_path='results.pdf', axes_style='white', x_jitter=[None, .2], 
                               subject='sub-wlsubj045', size=8)#, stimulus_superclass=['circular', 'radial', 'forward spiral'])
    g.set_ylabels('Preferred period (dpc)')
    g.set_xlabels('Eccentricity (deg)')
    g.set_yticklabels([0, '', 1, '', 2, '', 3, '', 4])
    g._legend.set_title("Stimulus class")
    for i, ax in enumerate(g.axes.flatten()):
        ax.set_title('Subject %s' % (i+1))
    g.savefig('results_onesubj.pdf')

In [ ]:
with sns.plotting_context('poster', font_scale=1.75):
    g = sfp.summary_plots.main(df, row=None, col='subject', task='task-sfpconstant', session=['ses-01', 'ses-02', 'ses-03'], y='preferred_period', varea=1, frequency_type='Local spatial frequency (cpd)', eccen_range=(1, 11),
                           plot_func=[sfp.plotting.plot_median_fit, sfp.plotting.scatter_ci_dist], xlim=(0,12), ylim=(0,4), axes_style='white', x_jitter=[None, .2],
                           subject='sub-wlsubj045', size=8, palette='Set2')#, stimulus_superclass=['circular', 'radial', 'forward spiral'])
    g.set_ylabels('Preferred period (dpc)')
    g.set_xlabels('Eccentricity (deg)')
    g.set_yticklabels([0, '', 1, '', 2, '', 3, '', 4])
    g._legend.set_title("Stimulus class")
    for i, ax in enumerate(g.axes.flatten()):
        ax.set_title('Subject %s' % (i+1))
    g.savefig('results2.pdf')

In [ ]:
# We want a reduced version of this to plot in comparison to the older studies, so we take the data presented in our summary plot and then look only at circular and radial
# I then take the median across bootstraps and the weighted average across subjects (the weight is the inverse of the size of the 68% confidence interval)
final_df = df[(df['task']=='task-sfp')&(df.varea==1)&(df.session.isin(['ses-01', 'ses-02', 'ses-03']))&(df.frequency_type=='Local spatial frequency (cpd)')&(df.eccen<11)]
final_df = final_df[['eccen', 'preferred_period', 'stimulus_superclass', 'subject', 'bootstrap_num']]
tmp_df = final_df.groupby(['subject', 'stimulus_superclass', 'eccen'])['preferred_period'].apply(np.percentile, [16, 84]).apply(lambda x: abs(x[1]-x[0])).reset_index(name='precision')
final_df = final_df.groupby(['eccen', 'stimulus_superclass', 'subject'])['preferred_period'].median().reset_index(name='preferred_period')
final_df = final_df.set_index(['subject', 'stimulus_superclass'])
final_df['precision'] = tmp_df.groupby(['subject', 'stimulus_superclass']).precision.mean()
final_df = final_df.reset_index()
final_df = final_df.groupby(['stimulus_superclass', 'eccen']).apply(lambda x: np.average(x.preferred_period, weights=1./x.precision)).reset_index(name='preferred_period')

In [ ]:
from sklearn import linear_model

circ_model = linear_model.LinearRegression(True)
tmp_df = final_df[final_df.stimulus_superclass=='circular']
circ_model.fit(tmp_df.eccen.values.reshape(-1, 1), tmp_df.preferred_period)
print("Circular stimuli")
print("score %s"%circ_model.score(tmp_df.eccen.values.reshape(-1, 1), tmp_df.preferred_period))
print("Coefficient: %s, Intercept: %s" % (circ_model.coef_, circ_model.intercept_))

rad_model = linear_model.LinearRegression(True)
tmp_df = final_df[final_df.stimulus_superclass=='radial']
rad_model.fit(tmp_df.eccen.values.reshape(-1, 1), tmp_df.preferred_period)
print("\nRadial stimuli")
print("score %s"%rad_model.score(tmp_df.eccen.values.reshape(-1, 1), tmp_df.preferred_period))
print("Coefficient: %s, Intercept: %s" % (rad_model.coef_, rad_model.intercept_))

# These values come off the Freeman and Simoncelli 2011 plot, grabbed using WebPlotDigitizer
V1_RF = np.array([[7.014, 1.199], [10.062, 1.815], [15.950, 2.740], [20.341, 3.510], [26.591, 4.538], [33.048, 5.565], [38.161, 6.490], [44.824, 7.568], [49.783, 8.339]])
rf_model = linear_model.LinearRegression(False)
rf_model.fit(V1_RF[:, 0].reshape(-1, 1), V1_RF[:, 1])
print("\nV1 RF size (Freeman and Simoncelli, 2011)")
print("score %s"%rf_model.score(V1_RF[:, 0].reshape(-1, 1), V1_RF[:, 1]))
print("Coefficient: %s, Intercept: %s" % (rf_model.coef_, rf_model.intercept_))

# From Benson et al, HCP 7T retinotopy data. This does have a non-zero intercept, not sure how to do that
# This is one sigma
V1_pRF_sigma = .063485
# For our purposes, we want to know the number of periods we have in a circle with 2 sigmas. For this to work, we need V1_pRF_sigma*2*2 (to get diameter, and then two sigmas)
V1_pRF_coef = V1_pRF_sigma * 4
print("\nV1 pRF size (Benson et al, 2018)")
print("Coefficient: %s" % V1_pRF_coef)

print("\nUsing pRF:")
print("Radial stimuli have %s periods per pRF" % (V1_pRF_coef / rad_model.coef_))
print("Circular stimuli have %s periods per pRF" % (V1_pRF_coef / circ_model.coef_))

In [ ]:
_=sfp.plotting.period_summary_plot(final_df, save_path='periods.svg')

In [ ]:
final_df = final_df.rename(columns={'eccen': "Eccentricity", 'stimulus_superclass': "Paper", 'preferred_period': 'Preferred Period (dpc)'})

In [ ]:
with sns.plotting_context('poster', font_scale=1.75), sns.axes_style('white'):
    g = sns.FacetGrid(final_df, hue='Paper', size=8, palette='deep', xlim=(0, 11))
    g.map_dataframe(sfp.plotting.scatter_ci_dist, 'Eccentricity', 'Preferred Period (dpc)', ci_vals=[50,50], x_jitter=True,marker='o', linewidth=6)
    g.map_dataframe(sfp.plotting.plot_median_fit, 'Eccentricity', 'Preferred Period (dpc)', linear_model.LinearRegression(True), linewidth=6, x_vals=(0, 10.5), linestyle='--')
    g.map_dataframe(sfp.plotting.plot_median_fit, 'Eccentricity', 'Preferred Period (dpc)', linear_model.LinearRegression(True), linewidth=6)
    g.set(yticks=[0, 1, 2])
    g.add_legend()
    g._legend.set_title('Stimulus class')
    g.savefig("final_summary.pdf")

In [ ]:
final_df.Paper = final_df.Paper.apply(lambda x: "This study, %s stimuli"%x)

data_dict = {
    'Paper': ['Sasaki (2001)',]*7,
    'Optimum SF (cpd)': [1.25, .9, .75, .7, .6, .5, .4],
    'Eccentricity': [0, 1, 2, 3, 4, 5, 12]
}
data_dict['Paper'].extend(['Henriksson (2008)', ]*5)
data_dict['Optimum SF (cpd)'].extend([1.2, .68, .46, .40, .18])
data_dict['Eccentricity'].extend([1.7, 4.7, 6.3, 9, 19])

data_dict['Paper'].extend(['Kay (2011)']*5)
data_dict['Optimum SF (cpd)'].extend([4, 3, 10, 10, 2])
data_dict['Eccentricity'].extend([2.5, 4, .5, 1.5, 7])

data_dict['Paper'].extend(["Hess (2009)"]*3)
data_dict['Optimum SF (cpd)'].extend([2.25, 1.9, 1.75])
data_dict['Eccentricity'].extend([2.5, 5, 10])

data_dict['Paper'].extend([ "D'Souza (2016)",]*3)
data_dict['Optimum SF (cpd)'].extend([2, .95, .4])
data_dict['Eccentricity'].extend([1.4, 4.6, 9.8])

data_dict['Paper'].extend(['Farivar (2017)']*2)
data_dict['Optimum SF (cpd)'].extend([3, 1.5,])
data_dict['Eccentricity'].extend([.5, 3])

data_dict['Paper'].extend([ 'Olsson (pilot, model fit)']*10)
data_dict['Optimum SF (cpd)'].extend([2.11, 1.76, 1.47, 2.75, 1.24, 1.06, .88, .77, .66, .60])
data_dict['Eccentricity'].extend([2, 3, 4, 1, 5, 6, 7, 8, 9, 10])

# Predictions of the scaling hypothesis
ecc = np.linspace(.01, 20, 50)
fovea_cutoff = 0
V1_RF_size = np.concatenate([np.ones(len(ecc[ecc<fovea_cutoff])), np.linspace(1, 2.5, len(ecc[ecc>=fovea_cutoff]))])
V1_RF_size = .2 * ecc

background_df = pd.DataFrame(data_dict)
background_df = background_df.sort_values(['Paper','Eccentricity',])
background_df["Preferred Period (dpc)"] = 1. / background_df['Optimum SF (cpd)']
background_df = pd.concat([background_df, final_df])

In [ ]:
pal = sns.color_palette('Set3', background_df.Paper.nunique()-2)
deep_pal = sns.color_palette('deep', 4)
keys = background_df['Paper'].unique()
pal = dict((k, p) for k, p in zip(keys, pal))
# so they correspond to the colors in the earlier plot
pal['This study, circular stimuli'] = deep_pal[0]
pal['This study, radial stimuli'] = deep_pal[2]
# pal['Olsson (pilot, model fit)'] = sns.color_palette('Blues', 3)[-1]
# pal['This study'] = sns.color_palette('Reds', 3)[-1]
with sns.plotting_context('poster', font_scale=1.75), sns.axes_style('white'):
    g = sns.FacetGrid(background_df, hue='Paper', size=8, aspect=1.25, palette=pal, hue_kws={'linewidth': [3]*7+[6]*2})
    g.map(sns.plt.plot, 'Eccentricity', 'Preferred Period (dpc)', marker='o')
    # g.map(plt.plot, 'Eccentricity', 'Preferred Period (dpc)', marker='o',)
    g.ax.set_ylim((0, 4))
    g.ax.set_xlim((0, 15))
    g.add_legend()
    #g.ax.legend(loc='best')
    g.set(xticks=[0, 4, 8, 12])
    g.set(yticks=[0, 1, 2, 3, ])
    g.ax.set_title("Summary of human V1 fMRI results")
    g.ax.set_xlabel('Eccentricity (deg)')
    g.savefig('SummaryWithData.pdf')

In [ ]:
first_level_df = full_ds['df']
first_level_df = first_level_df[(first_level_df.varea==1)&(first_level_df.eccen.isin(['02-03', '05-06']))&(first_level_df.stimulus_superclass.isin(['radial', 'circular']))]

In [ ]:
palette = [sns.color_palette("Reds", 10)[1]] + [sns.color_palette("Reds", 10)[4]]
with sns.axes_style('white'), sns.plotting_context("poster", font_scale=1.9):
    g = sns.FacetGrid(first_level_df, hue='eccen', palette=palette, size=6.5, col='stimulus_superclass', ylim=(0,4), xlim=(2**-3, 2**3.5), aspect=1.2,)
    #g.map_dataframe(sfp.plotting.plot_median, "Local spatial frequency (cpd)", 'amplitude_estimate', linewidth=6)
    g.map_dataframe(sfp.plotting.scatter_ci_dist, 'Local spatial frequency (cpd)', 'amplitude_estimate', linewidth=6)
    for ax in g.axes.flatten():
        ax.set_xscale('log', basex=2)
    g.set_yticklabels([0, '', 1, '', 2, '', 3, '', 4])
    g.set(xticks=[2**-3, 2**-1, 2**1, 2**3])
    g.add_legend()
    g._legend.set_title("Eccentricity")
    g.set_ylabels("Amplitude estimate")
    g.set_titles("Stimulus class = {col_name}")
    g.savefig('data.pdf')

In [ ]:
tuning_curve = ds['tuning_df']
tuning_curve = tuning_curve[(tuning_curve.varea==1)&(tuning_curve.eccen.isin(['02-03', '05-06']))&(tuning_curve.stimulus_superclass.isin(['radial', 'circular']))&(tuning_curve.frequency_type=="Local spatial frequency (cpd)")]

In [ ]:
palette = [sns.color_palette("Reds", 10)[1]] + [sns.color_palette("Reds", 10)[4]]
with sns.axes_style('white'), sns.plotting_context('poster', font_scale=1.9):
    g = sns.FacetGrid(tuning_curve, hue='eccen', palette=palette, size=6.5, col='stimulus_superclass', xlim=(2**-3, 2**3.5), ylim=(0,4), aspect=1.2)
    g.map_dataframe(sfp.plotting.plot_tuning_curve, linewidth=6)
    g.map(sns.plt.scatter, 'frequency_value', 'amplitude_estimate', linewidth=6)
    g.set_yticklabels([0, '', 1, '', 2, '', 3, '', 4])
    g.set(xticks=[2**-3, 2**-1, 2**1, 2**3])
    g.add_legend()
    g._legend.set_title("Eccentricity")
    g.set_xlabels("Local spatial frequency (cpd)")
    g.set_ylabels("Amplitude estimate")
    g.set_titles("Stimulus class = {col_name}")
    g.savefig('tuning_curves.pdf')

In [ ]:
stim_df = ds['stim_df']
stim_df[stim_df.w_a==-stim_df.w_r].w_a.unique()

So we can make higher res versions of the max frequency stimuli

In [ ]:
from scipy import misc as smisc
stim = sfp.stimuli.log_polar_grating(1080*4, 91, 91)
stim = sfp.utils.mask_array_like_grating(ds['stim'][0], stim)

In [ ]:
fig, ax = sns.plt.subplots(figsize=(20,20))
sfp.plotting.im_plot(stim, ax=ax)
fig.savefig('stim3.svg', bbox_inches='tight')

In [ ]:
sfp.plotting.stimuli(ds['stim'], ds['stim_df'], w_r=[0], w_a=[-4,-11,-32], figsize=(60, 20), save_path='stim7.svg')

In [ ]:
sfp.plotting.stimuli(ds['stim'], ds['stim_df'], w_x=[0, .004725,  .003341, .003341], w_y=[.004725, 0, .003341, -.003341], figsize=(80, 20), save_path='constant_stim.pdf')

# Analyzing the first-level results

After running our GLM analysis, we have the estimated amplitude responses of each voxel to each image class. Along with Noah Benson's anatomical template / Bayesian model, we also have each voxel's visual area and location in the visual field (in terms of eccentricity and polar angle). By combining the information contained within them, along with the dataframe describing each stimulus class, we can construct our tuning curves.

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import itertools
import pyPyrTools.JBhelpers as jbh
import pyPyrTools as ppt
import scipy as sp
from matplotlib.colors import LinearSegmentedColormap 

/home/billbrod/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/billbrod/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
# d = sfp.utils.load_data('wlsubj001', 'pilot01', atlas_type='prior', df_mode='full')
ds = sfp.utils.load_data('wlsubj045', 'ses-02', atlas_type='posterior', df_mode='summary', mat_type='stim_class', eccen_bin=True)
full_ds = sfp.utils.load_data('wlsubj045', 'ses-02', atlas_type='posterior', df_mode='full', mat_type='stim_class')
# ds2 = sfp.utils.load_data('wlsubj001', 'ses-pilot01', atlas_type='prior', df_mode='summary',mat_type='stim_class')

In [ ]:
tuning_df = ds['tuning_df']
tuning_df = tuning_df.drop_duplicates(['varea', 'eccen', 'stimulus_superclass', 'frequency_type'])
tuning_df = tuning_df.assign(subject='sub-wlsubj001', session='ses-pilot01', atlas_type='prior')
tuning_df['eccen'] = tuning_df.eccen.apply(lambda x: np.mean([float(i) for i in x.split('-')]))
tuning_df = tuning_df.reset_index(drop=True)
tuning_df['tuning_curve_peak'] = np.log2(tuning_df.tuning_curve_peak)

In [ ]:
melted_df = pd.melt(tuning_df, ['varea', 'eccen', 'stimulus_superclass', 'frequency_type', 'subject', 'session', 'atlas_type'], ['preferred_period','tuning_curve_peak'])

In [ ]:
tuning_df.head()

In [ ]:
g = sns.FacetGrid(tuning_df, 'frequency_type', 'varea', hue='stimulus_superclass', aspect=1, size=5, sharey='row')
g.map(sns.plt.plot, 'eccen', 'tuning_curve_peak')
g.set_titles("{row_name} | V{col_name}")
g.add_legend()

Here we see the different stimulus classes, as plotted in frequency space, colored by their superclass. These numbers are roughly log-spaced (doubling).

We also, in order to double-check some of our calculations, plot the distance and angle in frequency space: they look exactly correct.

In [ ]:
sfp.plotting.stimuli_properties(ds['df'])

In [ ]:
sfp.plotting.local_spatial_frequency(ds['df'])

In [ ]:
_ = sfp.plotting.stimuli_linear_approximation(d['stim'], d['stim_df'], d['stim_type'], 11, freq_space_distance=181, phi=np.pi/4, stimulus_superclass='radial', save_path='test.svg')

In [ ]:
Rmin, Rmax = sfp.first_level_analysis.find_ecc_range_in_degrees(stim[0,:,:], 12)
print("Inside radius of stimulus annulus: %.02f" % Rmin)
print("Outside radius of stimulus annulus: %.02f" % Rmax)

In [ ]:
sfp.plotting.check_hypotheses_with_data(d['tuning_df'], 'test_%s.svg')

In [ ]:
sfp.plotting.check_hypotheses_normalized(ds['tuning_df'], 'test_%s.svg')

In [ ]:
g=sns.FacetGrid(ds['tuning_df'], hue='eccen', row='frequency_type', palette='Reds', col='stimulus_superclass', sharey=False, size=4)
g.map(sns.plt.scatter, 'tuning_curve_bandwidth','tuning_curve_peak')


In [ ]:
plot_data(d['df'], 'freq_space_distance',)# ci_vals=[16, 84])

In [ ]:
tmp_df = d['df'][d['df'].stimulus_superclass=='circular']
g = sns.FacetGrid(tmp_df, hue='eccen', palette='Reds', size=5,)
#g.map(sns.regplot, 'rounded_freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'rounded_freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes.flatten():
    ax.set_xscale('log', basex=2)
g.add_legend()

If there were no scaling in the visual system, such that neurons at different places in the visual field were expected to have similar properties, the bottom would all line up well, and it doesn't!

In [ ]:
classes_of_interest = []
classes_of_interest.extend(df[(df.stimulus_superclass=='circular')&(df.rounded_freq_space_distance==11)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='radial')&(df.rounded_freq_space_distance==64)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='forward spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='reverse spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
#classes_of_interest.append(df[df.stimulus_superclass=='mixtures'].stimulus_class.values[0])

stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

tmp_df = df[df.stimulus_superclass=='mixtures']
g = sns.FacetGrid(tmp_df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

In [ ]:
angles = sorted(df.freq_space_angle.unique())
stim_idxs = []
for ang in angles:
    class_of_interest = df[(df.freq_space_angle==ang)&(df.rounded_freq_space_distance==32)].stimulus_class.unique()[0]
    stim_idxs.append(stim_df[stim_df.class_idx==class_of_interest].index[0])

#stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
tmp_df = pd.DataFrame(df.groupby(['eccen_bin', 'w_r', 'w_a']).modelmd.mean()).reset_index()

g = sns.FacetGrid(tmp_df, col='eccen_bin', col_wrap=4, size=5)
cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
g.map(sfp.utils.scatter_heat, 'w_a', 'w_r', 'amplitude_estimate_median', vmin=tmp_df['amplitude_estimate_median'].min(), 
      vmax=tmp_df['amplitude_estimate_median'].max())
sns.plt.colorbar(cax=cbar_ax)
g.fig.subplots_adjust(right=.9, top=.9)
g.fig.suptitle('Average response amplitude estimates at each point in frequency space')

In [ ]:
tmp_df = pd.pivot_table(df, 'amplitude_estimate_median', 'rounded_freq_space_distance', 'eccen_bin')
norm_df = tmp_df.copy()
for col in norm_df.columns:
    norm_df[col] = norm_df[col] / norm_df[col].max()

In [ ]:
fig = sns.heatmap(tmp_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Average response amplitude estimates')

In [ ]:
fig = sns.heatmap(norm_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Normalized average response amplitude estimates')

In [ ]:
df = pd.read_csv('/home/billbrod/Data/spatial_frequency_preferences/derivatives/first_level_analysis/stim_class/prior/sub-wlsubj001/ses-pilot01/sub-wlsubj001_ses-pilot01_task-sfp_summary_v1_e2-8_eccen_bin_hemi_bin.csv')

In [ ]:
sns.distplot(df.R2.values)

# Create plots for first year talk

In [ ]:
# for this, we're only using circular results
tmp_df = df[df.stimulus_superclass=='circular']
# tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num']]

In [ ]:
tmper_df = tmp_df[tmp_df.freq_space_distance==6.]
tmper_df = tmper_df.groupby(['eccen', 'bootstrap_num'])['amplitude_estimate'].mean().unstack().reset_index()

In [ ]:
hyp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num'], var_name='Frequency')
hyp_df = pd.DataFrame(hyp_df.groupby(['Frequency', 'value', 'bootstrap_num'])['amplitude_estimate'].mean()).reset_index()

In [ ]:
#print("freq_space_distance min: %.03f, max: %.03f" % (tmp_df.freq_space_distance.min(), tmp_df.freq_space_distance.max()))
#print("Halfway in log space: %.03f" % np.floor(2**((np.log2(181.) + np.log2(6.))/2.)))
# because these are circular, freq_space_distance==w_r
#stim_idx = stim_df[(stim_df.w_r.isin([6, 32, 181]))&(stim_df.w_a==0)].index[::8]
stim_idx = d['stim_df'][(d['stim_df'].w_r.isin([6, 32, ]))&(d['stim_df'].w_a==0)].index[::8]
stims = [d['stim'][i] for i in stim_idx]

We actually want to plot windows of the stimuli instead of just sins

In [ ]:
max_degree_rad = 12
scale_factor = 10
mask = sfp.utils.create_circle_mask(750, 350, scale_factor* 1080/(2*2*max_degree_rad), 1080)
stim_windows = [mask * s + ~mask.astype(bool)*127 for s in stims]


In [ ]:
tmp_df = d['df']
tmp_df = tmp_df[(tmp_df.stimulus_superclass=='circular')&(tmp_df.eccen.isin(['02-03', '03-04', '04-05', '05-06', '06-07', '07-08']))]#&(tmp_df.bootstrap_num.isin(range(5)))]
sfp.plotting.compare_hypotheses(tmp_df)

In [ ]:
tmp_df = d['df']
tmp_df = tmp_df[(tmp_df.bootstrap_num.isin(range(5)))]
sfp.plotting.compare_hypotheses_talk(tmp_df, stims)

In [ ]:
# for this, we're only using circular results
tmp_df = d['df'][d['df'].stimulus_superclass.isin(['circular'])]
# tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
tmp_df = tmp_df[['stimulus_superclass','eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num']]
tmp_df = tmp_df[tmp_df.bootstrap_num.isin(range(5))]

peak_df = sfp.plotting.peak_spatial_frequency(tmp_df, id_vars=['stimulus_superclass'])

In [ ]:
speak_df

In [ ]:
sns.plt.scatter(bandwidth[:,0], bandwidth[:,1], c=sns.color_palette('Reds', 6), s=75)

In [ ]:
ecc = np.linspace(.01, 9, 50)
RF_scale_factor = 4.
V1_RF_size = np.concatenate([np.ones(len(ecc[ecc<.5]))/RF_scale_factor, np.linspace(1/RF_scale_factor, 4/RF_scale_factor, len(ecc[ecc>=.5]))])
#V2_RF_size = np.concatenate([2*np.ones(len(ecc[ecc<4])), np.linspace(2, 2.5, len(ecc[ecc>=4]))])

Olsson_peak = [2.75, 2.11, 1.76, 1.47,1.24, 1.06, .88, .77, .66, .60]
Olsson_ecc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

with sns.plotting_context('poster', font_scale=1), sns.axes_style('white'):
    # because this doesn't represent data, just intuition, we use this.
#    with plt.xkcd():
    x = np.linspace(.01, 9, 50)
    y = []
    fig, axes = sns.plt.subplots(1,1, figsize=(13,6))
    ax = axes
    # this gives intuitive plots, currently we want the possible hypotheses instead
#         for i in range(3):
#             y.append(10/(x+2)+i)
#             ax.plot(x, y[-1],  label='V%s'%(i+1), color=['r','g','b'][i])
    colors = sns.color_palette(n_colors=3)
    for i, (p, o) in enumerate([[peak_01_s0, opt_a_01_s0], [peak_01, opt_a_01]]):#, [peak_45, opt_a_45]]):
#    ax.plot(ecc, hyperbola(ecc, opt_a), '-', label='scaling')
#        ax.plot(ecc, hyperbola(ecc, o), '-', label='scaling', c= colors[i])
        ax.plot(ecc, hyperbola(ecc, o), '-', label='Subject %s'%(i+1), c= colors[i])
#    ax.plot(ecc, np.ones(len(ecc))*RF_scale_factor, '--', label='constant')
#        ax.set_ylim((0,8))
#    sns.plt.scatter(peak[:, 0], peak[:, 1], c=sns.color_palette('Reds', 6), s=75,)# label='This study')
        if i==1:
            sns.plt.scatter(p[:, 0]+.07, p[:, 1], c=colors[i], s=75,)# label='This study')
        else:
            sns.plt.scatter(p[:, 0], p[:, 1], c=colors[i], s=75,)# label='This study')
#    sns.plt.scatter(Olsson_ecc, Olsson_peak, s=75, c= sns.color_palette('Blues', 10), label='Olsson pilot')
    ax.set_xlabel("Receptive field center eccentricity (degrees)")
    ax.set_ylabel("Peak spatial frequency (cpd)")
    ax.set_title("")
    ax.set_ylim((.5, 4.5))
    ax.set_xlim((0, 11))

    sns.plt.legend(title="Subjects", loc='best')
#    ax.figure.savefig('results-hypotheses.svg', bbox_inches='tight')